### 기상청 날씨 데이터
* BeautifulSoup의 find(), find_all() 함수을 사용해서 tag 선택하기
* 파싱한 데이터를 List에 Dict에 저장하기

In [11]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

# <title> 태그 선택
print(type(soup.find('title')), soup.find('title').string)
print(type(soup.find_all('title')))
for title_tag in soup.find_all('title'):
    print(type(title_tag), title_tag.string)

200
<class 'bs4.element.Tag'> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'> 기상청 육상 중기예보
<class 'bs4.element.Tag'> 전국 육상 중기예보 - 2021년 04월 28일 (수)요일 06:00 발표
<class 'bs4.element.Tag'> 전국 육상중기예보
